# Single-Qubit Measurements

This kata introduces you to measurements done on single-qubit systems.

The concept of a measurement is a central part of quantum mechanics, as well as a key component of all quantum algorithms. Single-qubit measurements, as their name implies, are measurements done on single qubits. The outcomes of a measurement in quantum mechanics are probabilistic, and in general, change the state of the qubit according to the outcome of the measurement.

We recommend you complete the [Single-Qubit Gates kata](../SingleQubitGates/SingleQubitGates.ipynb) before starting this one.

**This kata covers the following topics:**

- Computational basis measurements
- Pauli basis measurements
- Measurements in arbitrary orthogonal bases
- Representing measurements as projector operators

**What you should know to start working on this kata:**

- Basic linear algebra
- The concept of a qubit
- Single-qubit gates

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from test_SingleQubitMeasurements import exercise

## Computational basis measurements

In this section, we will discuss the simplest type of qubit measurements - measurements in the computational basis. This is the "default" type of measurements. Unless otherwise specified, "measurement" refers to this type.

The state $\ket{\psi}$ of a single qubit can always be expressed in Dirac notation as:
$$\ket{\psi} = \alpha \ket{0} + \beta \ket{1}$$
where $\alpha$ and $\beta$ are complex numbers, and the state is normalized, $|\alpha|^2 + |\beta|^2 = 1$.

To get some information about the qubit state, we can measure the qubit. Similar to the classical case of measuring a bit, the outcome of a measurement can be $0$ or $1$. However, unlike the classical case, quantum measurement is a probabilistic process.
The probabilities of the measurement outcomes being $0$ and $1$ are $|\alpha|^2$ and $|\beta|^2$, respectively.

Additionally, the state of the qubit is modified by the measurement - if the outcome of the measurement is $0$, then the post-measurement state of the qubit is $\ket{0}$, and if the outcome is $1$, the state is $\ket{1}$. In quantum mechanics, this is referred to as the <a href="https://en.wikipedia.org/wiki/Wave_function_collapse" target="_blank">collapse of the wave function</a>.

The outcomes of measuring a qubit in the state $\alpha \ket{0} + \beta \ket{1}$ in the computational basis and their probabilities are summarized in the table below:

| Measurement outcome | Probability of outcome | State after measurement |
|---|---|---|
| $0$ | $\|\alpha\|^2$ | $\ket{0}$ |
| $1$ | $\|\beta\|^2$  | $\ket{1}$ |

>Unlike quantum gates, which are unitary and reversible operations, measurements are neither unitary nor reversible. Since the outcomes of a measurement are probabilistic, any two isolated qubits which are initially prepared in identical superposition states are in general not guaranteed to have the same measurement outcomes after each qubit has been measured separately. As we will see below, measurements are modeled by projection operators instead of unitary operators.
>
>Additionally, the assumption of the wave function being **normalized** is important, since the probability outcomes must sum up to $1$. If the wave function is not normalized, it is important to normalize it first in order to obtain the correct measurement probabilities.

### Analyze 1: Computational basis measurement

The qubit is in the following state:
$$\ket \psi = 0.6 \ket 0 + 0.8 \ket 1 = \begin{bmatrix} 0.6 \\ 0.8 \end{bmatrix}$$

If this qubit is measured in the computational basis, what are the outcome probabilities?

<details>
<summary><b>Solution</b></summary>

The given state $\ket \psi$ is normalized, since $0.6^2 + 0.8^2 = 1$. Hence, the probability of measuring $0$ is $|0.6|^2 = 0.36$, and the probability of measuring $1$ is $|0.8|^2 = 0.64$.
</details>

## Measurements in Qiskit

The following example demonstrates how to perform measurements in Qiskit. It prepares a qubit in the state $0.6\ket{0} + 0.8\ket{1}$ and then measures it in the computational basis.

In Qiskit, measurements are implemented as the `measure` method of the `QuantumCircuit` object. Measurement outcomes are also represented as a part of a circuit: you need to define a `ClassicalRegister` variable of length matching the length of the `QuantumRegister` you'll measure (in this case, of length $1$) and pass it as an extra argument when creating a `QuantumCircuit`.

After you run the circuit on a simulator, you can extract measurement outcomes using the `get_counts` method.

> If you run this code multiple times, you will notice that whenever the measurement outcome is $1$, the post-measurement state of the qubit is $\ket{1}$, and similarly for outcome $0$ the final state is $\ket{0}$. This is in line with our expectation that after the measurement the wave function "collapses" to the corresponding state.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

# Create the simulator instance
simulator = AerSimulator(method='statevector')

# Define the quantum circuit
qr = QuantumRegister(1)
cr = ClassicalRegister(1)
circ = QuantumCircuit(qr, cr)
# Initialize the qubit in the state 0.6|0⟩ + 0.8|1⟩
circ.initialize([0.6, 0.8], qr)
circ.save_statevector(label="Before measurement")

# Measure the qubit
circ.measure(qr, cr)
circ.save_statevector(label="After measurement")

# Run the simulation once and get the result
res = simulator.run(circ, shots=1).result()
# Extract the saved state vectors
labels = ["Before measurement", "After measurement"]
for label in labels:
    state_vector = res.data()[label]
    print(label)
    print(state_vector.data)
print(f"Measurement outcome: {list(res.get_counts().keys())[0]}")

## Measurement statistics

The following code demonstrates that the theoretical and experimental values of the probability outcomes indeed match with each other. We run the circuit from the previous example a thousand times and get the frequencies of the measurement outcomes. At the end, we expect $0$ to be measured approximately $|0.6 |^2 \cdot 1000 = 360$ times, and $1$ to be measured approximately $|0.8|^2 \cdot 1000 = 640$ times. Note that since measurements are probabilistic, we do not expect the results to match these values exactly. As we repeat the measurements, the resulting distribution will align with the theoretical probabilities.

In [ ]:
# Run the simulation multiple times and get the frequencies of different measurement outcomes
n_shots = 1000
res = simulator.run(circ, shots=n_shots).result()
counts = res.get_counts()
print(f"Simulated probability of measuring 0 is {counts['0'] / n_shots}.")
print("Theoretical probability of measuring 0 is 0.36.")
print(f"Simulated probability of measuring 1 is {counts['1'] / n_shots}.")
print("Theoretical probability of measuring 1 is 0.64.")

Measurements can be used to distinguish orthogonal states. We start with an exercise for distinguishing between the computational basis states, and discuss the general case of arbitrary basis measurements later in the kata.

## Exercise 1. Distinguish |0⟩ and |1⟩

**Inputs:** 

1. A quantum circuit.
2. A qubit which is guaranteed to be in either the $\ket{0}$ or the $\ket{1}$ state, represented as `QuantumRegister` of length 1.
3. A classical register of length 1.

**Goal:** Measure the qubit and write the result (`0` if the qubit was in the $\ket{0}$ state, or `1` if it was in the $\ket{1}$ state) to the classical register. The state of the qubit at the end of the operation does not matter.

In [ ]:
@exercise
def zero_or_one(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    # Write your code here
    ...

## Measurements in the Pauli bases

So far, we have discussed measurements done in the computational basis, that is, the $\{ \ket{0}, \ket{1}\}$ basis.

It is also possible to implement measurements in other orthogonal bases, such as the Pauli X basis, which consists of the two vectors $\ket{+} = \tfrac1{\sqrt2} (\ket{0} + \ket{1})$ and $\ket{-} = \tfrac1{\sqrt2} (\ket{0} - \ket{1})$.

The eigenvalues of a Pauli matrix are $\pm 1$, with one eigenvector corresponding to each eigenvalue. For any chosen Pauli basis, the measurement in that basis returns $0$ if the measurement outcome corresponds to the eigenvalue $+1$, and returns $1$ if the measurement outcome corresponds to the eigenvalue $-1$. As in the case of the computational basis measurements, the wave function of the qubit collapses to the corresponding state after the measurement is executed.

The probabilities of the outcomes are defined using a similar rule: to measure a state $\ket{\psi}$ in a Pauli basis $\{ \ket{b_0}, \ket{b_1} \}$, we represent it as a linear combination of the basis vectors:
$$\ket{\psi} = c_0 \ket{b_0} + c_1 \ket{b_1}$$

The probabilities of outcomes $\ket{b_0}$ and $\ket{b_1}$ will be defined as $|c_0|^2$, and $|c_1|^2$, respectively.

> Computational basis measurement is often referred to as measurement in Pauli Z basis. Indeed, the eigenvectors of the Z gate are $\ket{0}$ and $\ket{1}$, with eigenvalues $+1$ and $-1$, respectively.

## Exercise 2. Distinguish |+⟩ and |-⟩

**Inputs:** 

1. A quantum circuit.
2. A qubit which is guaranteed to be in either the $\ket{+}$ or the $\ket{-}$ state, represented as `QuantumRegister` of length 1.
3. A classical register of length 1.

**Goal:** Measure the qubit and write the result (`0` if the qubit was in the $\ket{+}$ state, or `1` if it was in the $\ket{-}$ state) to the classical register. The state of the qubit at the end of the operation does not matter.

In [ ]:
@exercise
def plus_or_minus(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    # Write your code here
    ...

## Measurements in arbitrary orthogonal bases

It is possible to measure a qubit in orthogonal bases other than the Pauli bases. Suppose one wants to measure a qubit in an orthonormal basis $\ket{b_0}$ and $\ket{b_1}$. Let the state of the qubit be represented by the normalized vector $\ket{\psi}$. Then, one can always express the state in terms of the basis vectors $\ket{b_0}$ and $\ket{b_1}$, i.e., there exist complex numbers $c_0, c_1$, such that
$$
\ket{\psi} = c_0 \ket{b_0} + c_1 \ket{b_1}.
$$
The rule for obtaining the probabilities of measurement outcomes is exactly the same as that for the computational basis measurement. For a measurement in the $\{ \ket{b_0}, \ket{b_1}\}$ basis we get

- Outcome $b_0$ with probability $|c_0|^2$ and the post-measurement qubit state of $\ket{b_0}$
- Outcome $b_1$ with probability $|c_1|^2$ and the post-measurement qubit state of $\ket{b_1}$

This can be summarized in the following table:

| Measurement outcome | Probability of outcome | State after measurement |
|---|---|---|
| $b_0$ | $\|c_0\|^2$ | $\ket{b_0}$ |
| $b_1$ | $\|c_1\|^2$  | $\ket{b_1}$ |

As before, the assumption of $\ket \psi$ being normalized is important, since it guarantees that the two probabilities add to $1$.

> As you might recall, a global phase is said to be hidden or unobservable.
This is explained by the fact that global phases have no impact on quantum measurements. For example, consider two isolated qubits which are in (normalized) states $\ket \psi$ and $e^{i\theta}\ket \psi$.
If both are measured in an orthogonal basis $\{ \ket{b_0},\ket{b_1}\}$, the probabilities of measuring $b_0$ or $b_1$ are identical in both cases, since $|\bra{b_i}\ket{\psi}|^2 = |\bra{b_i}e^{i\theta}\ket{\psi}|^2  $.
Similarly, for either qubit, if $b_i$ is the measurement outcome, the post-measurement state of the qubit is $\ket{b_i}$. Hence, the measurements are independent of the global phase $\theta$.

### Measurements as projection operations

Just as qubits are represented by vectors and quantum gates are represented by matrices, quantum measurements are represented by _orthogonal projection operators_. An orthogonal projection operator (sometimes referred to as _projector_) is a matrix $P$ which satisfies the following property:
$$
P^2 = P^\dagger = P.
$$
(As usual, the $\dagger$ symbol denotes conjugate transposition.)

As the name implies, orthogonal projection operators project the state of the qubit onto an orthogonal subspace. Using the ket-bra representation, one can represent a projection matrix in the Dirac notation.
For example, one may construct a projector onto the $\ket{0}$ subspace as:
$$
P = \ket 0 \bra 0 \equiv \begin{bmatrix} 1 & 0 \\ 0 & 0 \end{bmatrix}.
$$

A measurement in an orthogonal basis $\{ \ket{b_0}, \ket{b_1}\}$ is described by a pair of projectors $P_0 = \ket{b_0}\bra{b_0}$ and $P_1 = \ket{b_1}\bra{b_1}$. Since $\ket{b_0}$ and $\ket{b_1}$ are orthogonal, their projectors are also orthogonal, i.e., $P_0 P_1 = P_1 P_0 = 0$. The rules for measurements in this basis can then be summarized as follows:

- Measuring a qubit in a state $\ket \psi$ is done by picking one of these projection operators at random.
- Projector $P_0$ is chosen with probability $|P_0 \ket{\psi}|^2$, and the projector $P_1$ is chosen with probability $|P_1\ket{\psi}|^2.$
- If projector $P_0$ is chosen, the post-measurement state of the qubit is given by
    $$
    \frac1{|P_0 \ket{\psi}|}P_0 \ket\psi,
    $$
    and similarly for $P_1$.

Although this formalism looks different from the previous sections, it is in fact equivalent. If $\ket \psi = c_0 \ket{b_0} + c_1 \ket{b_1}$, we have
$$
P_0 \ket \psi = c_0 \ket{b_0}, \text{so that } | P_0\ket \psi| = |c_0|,
$$
and similarly,
$$
P_1 \ket \psi = c_1 \ket{b_1}, \text{so that } |P_1\ket \psi| = |c_1|.
$$

Thus, as before, the probability of measuring $b_0$ is $|P_0\ket\psi|^2 = |c_0|^2$, and the probability of measuring $b_1$ is $|P_1\ket\psi|^2 = |c_1|^2$. Similarly, one can verify that the post-measurement outcomes are also $\ket{b_0}$ and $\ket{b_1}$, respectively (up to unobservable global phases).

Although the projector formalism for single-qubit systems may seem superfluous, its importance will become clear later, when we consider measurements for multi-qubit systems.

## Implementing arbitrary basis measurements

In exercise 2, we used a Hadamard gate together with a measurement in the computation basis to do a measurement in a Pauli X basis. We will now show that it is always possible to measure a qubit in any orthogonal basis using just unitary rotation matrices and computation basis measurements.

Consider a state $\ket{\psi} = c_0 \ket{b_0} + c_1 \ket{b_1}$ which we would like to measure in an orthonormal basis $\{ \ket{b_0}, \ket{b_1}\}$. First, we construct the following unitary matrix:
$$
U = \ket{0} \bra{b_0} + \ket{1} \bra{b_1}
$$

The conjugate transpose of this unitary is the operator
$$
U^\dagger = \ket{b_0} \bra{0} + \ket{b_1} \bra{1}
$$

(One may verify that $U$ is indeed a unitary matrix, by checking that $U^\dagger U = U U^\dagger = I$)

Note that the effect of these matrices on the two bases is the following:
$$U\ket{b_0} = \ket{0}$$
$$U\ket{b_1} = \ket{1}$$
$$U^\dagger \ket{0} = \ket{b_0}$$
$$U^\dagger \ket{1} = \ket{b_1}$$

In order to implement a measurement in the $\{ \ket{b_0}, \ket{b_1} \}$ basis, we do the following:

1. Apply $U$ to $\ket{\psi}$.  
   The resulting state is $U\ket{\psi} = c_0 \ket{0} + c_1 \ket{1} $.
2. Measure the state $U\ket{\psi}$ in the computational basis.  
   The outcomes $0$ and $1$ occur with probabilities $|c_0|^2$ and $|c_1|^2$.
3. Apply $U^\dagger$ to the post-measurement state.  
   This transforms the states $\ket{0}$ and $\ket{1}$ to the states $\ket{b_0}$ and $\ket{b_1}$, respectively.

Thus, $b_0$ and $b_1$ are measured with probabilities $|c_0|^2$ and $|c_1|^2$, respectively, with the end state being $\ket{b_0}$ and $\ket{b_1}$ - which is exactly the measurement we want to implement.

This procedure can be used to distinguish arbitrary orthogonal states as well, as will become clear from the following exercises.

### Analyze 2: Pauli X basis measurement

What are the outcome probabilities of measuring a qubit in the $\ket{0}$ state in the Pauli X basis, i.e., the $\{ \ket{+}, \ket{-}\}$ basis?

<details>
<summary><b>Solution</b></summary>

To find the probabilities of measuring $+$ and $-$, we first need to express the state $\ket{0}$ in terms of $\ket{+}$ and $\ket{-}$. Using the fact that $\ket{\pm} = \tfrac{1}{\sqrt{2}} (\ket{0} \pm \ket{1})$, we can show that
    $$
    \ket{0} = \tfrac{1}{\sqrt{2}} \ket{+} + \tfrac{1}{\sqrt{2}} \ket{-}.
    $$
Thus, the probability of measuring $+$ is $|\frac1{\sqrt2}|^2 = 0.5$, and similarly, the probability of measuring $-$ is $0.5$.
</details>

### Analyze 3: Pauli Y basis measurement

What are the outcome probabilities of measuring a qubit in the $0.6\ket{0} + 0.8\ket{1}$ state in the Pauli Y basis, i.e., the $\{ \ket{i}, \ket{-i}\}$ basis?

<details>
<summary><b>Solution</b></summary>

Similar to the previous analysis exercise, we need to express the state $\ket{\psi} = 0.6 \ket{0} + 0.8 \ket{1}$ in the $\ket{\pm i}$ basis. For this calculation, we use the projection matrix approach.

First, we recall that the states $\ket{\pm i}$ are given by
    $$
    \ket{\pm i} = \tfrac1{\sqrt2} (\ket{0} \pm i \ket{1}).
    $$

We can now construct the two projectors $P_{\pm i}$ onto states $\ket {\pm i}$ as follows:
    $$P_{i} = \ket{i}\bra{i} = \tfrac{1}{2} \begin{bmatrix} 1 \\ i \end{bmatrix} \begin{bmatrix} 1 & -i \end{bmatrix} = \tfrac{1}{2} \begin{bmatrix}1 & -i \\ i & 1 \end{bmatrix}$$
    $$P_{-i} = \ket{-i}\bra{-i} = \tfrac{1}{2} \begin{bmatrix} 1 \\ -i \end{bmatrix} \begin{bmatrix} 1 & i \end{bmatrix} = \tfrac{1}{2} \begin{bmatrix}1 & i \\ -i & 1 \end{bmatrix}$$

Recalling that the probabilities of measuring $\pm i$ are equal to the norm of the vectors $P_{\pm i}\ket{\psi}$, we now apply $P_{\pm i}$ to $\ket{\psi}$:
    $$P_{+i} \ket{\psi} = \tfrac{1}{2} \begin{bmatrix}1 & -i \\ i & 1\end{bmatrix} \begin{bmatrix} 0.6 \\ 0.8 \end{bmatrix} = \tfrac{1}{2} \begin{bmatrix} 0.6 - 0.8i \\ 0.8 + 0.6i \end{bmatrix}$$
    $$P_{-i} \ket{\psi} = \tfrac{1}{2} \begin{bmatrix}1 & i \\ -i & 1\end{bmatrix} \begin{bmatrix} 0.6 \\ 0.8 \end{bmatrix} = \tfrac{1}{2} \begin{bmatrix} 0.6 + 0.8i \\ 0.8 - 0.6i \end{bmatrix}$$

Hence, the probabilities of measuring $\pm i$, which we denote by $p(\pm i)$, are:
    $$p(+i) = |P_{+i} \ket{\psi}|^2 = \tfrac{1}{4}(|0.6 - 0.8i|^2 + |0.8 + 0.6i|^2) = \tfrac{1}{2}$$
    $$p(-i) = |P_{-i} \ket{\psi}|^2 = \tfrac{1}{4}(|0.6 + 0.8i|^2 + |0.8 - 0.6i|^2) = \tfrac{1}{2}$$

</details>


## Exercise 3. Distinguish fixed orthogonal states

**Inputs:** 

1. A quantum circuit.
2. A qubit which is guaranteed to be in either the $\ket{\psi_+} = 0.6\ket{0} + 0.8\ket{1}$ or $\ket{\psi_-} = -0.8\ket{0} + 0.6\ket{1}$ state, represented as `QuantumRegister` of length 1.
3. A classical register of length 1.

**Goal:** Measure the qubit and write the result (`0` if the qubit was in the $\ket{\psi_+}$ state, or `1` if it was in the $\ket{\psi_-}$ state) to the classical register. The state of the qubit at the end of the operation does not matter.

<details>
<summary><strong>Need a hint?</strong></summary>

A suitable $R_y$ rotation can be used to go from the computational basis $\{ \ket{0}, \ket{1} \}$ to the $\{ \ket{\psi_+}, \ket{\psi_-} \}$ basis and vice versa.
</details>

In [ ]:
@exercise
def psi_plus_or_psi_minus(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    # Write your code here
    ...

## Exercise 4. Distinguish parameterized orthogonal states

**Inputs:** 

1. A quantum circuit.
2. A qubit which is guaranteed to be in either the $\ket{A} = \cos \alpha \ket{0} - i \sin \alpha \ket{1}$ or $\ket{B} = - i \sin \alpha \ket{0} + \cos \alpha \ket{1}$ state, represented as `QuantumRegister` of length 1.
3. A classical register of length 1.
4. Angle $\alpha$, in radians, represented as `float`.

**Goal:** Measure the qubit and write the result (`0` if the qubit was in the $\ket{A}$ state, or `1` if it was in the $\ket{B}$ state) to the classical register. The state of the qubit at the end of the operation does not matter.

<details>
<summary><strong>Need a hint?</strong></summary>

A suitable $R_x$ rotation can be used to go from the computational basis $\{ \ket{0}, \ket{1} \}$ to the $\{ \ket{A}, \ket{B} \}$ basis and vice versa.
</details>

In [ ]:
@exercise
def a_or_b(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister, alpha: float) -> None:
    # Write your code here
    ...

# Conclusion

Congratulations! In this kata you learned to apply measurements to single-qubit systems. Here are a few key concepts to keep in mind:

- Unlike quantum gates, measurements are neither unitary nor reversible. When we measure a qubit, the state of the qubit collapses to one of the basis states, and the initial state is lost.
- Measurements are always done in an orthogonal basis. By default, we choose the computational basis $\{ \ket{0}, \ket{1} \}$.
- Measurements are represented as sets of projector operators, which are matrices.
- In Qiskit, you can implement measurements in the computational basis using the `measure` method, which takes a quantum register to be measured and a classical register to write measurement results to.